In [2]:
import os
import sys
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import time
import calendar
from datetime import datetime, date, timedelta
import pickle


In [3]:
#open the df_bitfinex_volume.csv file
df_bitfinex_volume = pd.read_csv("df_bitfinex_volume.csv")
df_bitfinex_volume.index =df_bitfinex_volume['Date']
df_bitfinex_volume = df_bitfinex_volume.drop(['Date'], axis=1)
df_bitfinex_volume

,Daily Volume
Date,
2013-03-31,5224.401313
2013-04-01,8376.527478
2013-04-02,12996.245072
2013-04-03,6981.668305
2013-04-04,5234.882203
...,...
2023-07-11,1636.401373
2023-07-12,3763.115501
2023-07-13,2982.608992


In [97]:
# open the df_price_btc_usd_agregate file
df_price_btc_usd_agregate = pd.read_csv("/home/moises/Data_BTC_thesis/df_price_btc_usd_agregate.csv")
df_price_btc_usd_agregate_svar = df_price_btc_usd_agregate[df_price_btc_usd_agregate["Date"]>="2013-03-31"]
df_price_btc_usd_agregate_svar = df_price_btc_usd_agregate_svar[df_price_btc_usd_agregate_svar["Date"]<="2023-07-15"] 
df_price_btc_usd_agregate_svar.index = df_price_btc_usd_agregate_svar['Date']
del df_price_btc_usd_agregate_svar['Date']
df_price_btc_usd_agregate_svar


,Close
Date,
2013-03-31,92.743611
2013-04-01,100.409370
2013-04-02,108.063618
2013-04-03,128.261101
2013-04-04,132.155428
...,...
2023-07-11,30371.646832
2023-07-12,31451.123508
2023-07-13,30309.254446


In [22]:
#get df_blockstats_daily_mean.csv
df_blockstats_daily_mean = pd.read_csv("/home/moises/Data_BTC_thesis/df_blockstats_daily_mean.csv")
df_blockstats_daily_mean.columns

Index(['GMT', 'index', 'avgfee', 'avgfeerate', 'avgtxsize', 'height', 'ins',
       'maxfee', 'maxfeerate', 'maxtxsize', 'medianfee', 'mediantime',
       'mediantxsize', 'minfee', 'minfeerate', 'mintxsize', 'outs', 'subsidy',
       'swtotal_size', 'swtotal_weight', 'swtxs', 'time', 'total_out',
       'total_size', 'total_weight', 'totalfee', 'txs', 'utxo_increase',
       'utxo_size_inc', 'utxo_increase_actual', 'utxo_size_inc_actual', 'year',
       'avgfee_btc', 'maxfee_btc', 'minfee_btc', 'subsidy_btc',
       'blockhash_decimal', 'Close', 'Date', 'avgfee_USD', 'maxfee_USD',
       'minfee_USD'],
      dtype='object')

In [23]:
df_blockstats_daily_mean["total_out_usd"] =(df_blockstats_daily_mean["total_out"] + df_blockstats_daily_mean["totalfee"] + df_blockstats_daily_mean["subsidy"])

In [21]:
#df_blockstats_daily_mean = df_blockstats_daily_mean[df_blockstats_daily_mean["Date"]>="2013-03-31"]
df_blockstats_daily_mean

,GMT,index,avgfee,avgfeerate,avgtxsize,height,ins,maxfee,maxfeerate,maxtxsize,...,maxfee_btc,minfee_btc,subsidy_btc,blockhash_decimal,Close,Date,avgfee_USD,maxfee_USD,minfee_USD,total_out_usd
1542,2013-03-31,879.875,97333.220238,163.267857,830.791667,228922.541667,707.571429,7.775612e+06,3338.738095,16464.940476,...,0.077756,5.952381e-07,25.00,2.082020e+60,92.743611,2013-03-31,0.090270,7.211384,0.000055,9.171621e+11
1543,2013-04-01,105.500,80666.772727,158.116162,570.424242,229106.500000,650.797980,2.725213e+06,5080.934343,15467.085859,...,0.027252,1.515152e-06,25.00,2.113253e+60,100.409370,2013-04-01,0.080997,2.736369,0.000152,7.318517e+11
1544,2013-04-02,295.000,81245.955801,161.950276,528.209945,229296.000000,811.618785,8.303811e+06,7573.016575,16302.441989,...,0.083038,1.104972e-06,25.00,1.948098e+60,108.063618,2013-04-02,0.087797,8.973399,0.000119,1.430121e+12
1545,2013-04-03,473.000,95721.051429,194.994286,533.354286,229474.000000,849.680000,1.400018e+07,10347.594286,14619.680000,...,0.140002,0.000000e+00,25.00,2.049811e+60,128.261101,2013-04-03,0.122773,17.956781,0.000000,1.355043e+12
1546,2013-04-04,637.500,98543.233766,192.428571,571.006494,229638.500000,1032.461039,1.963353e+07,6570.993506,18351.681818,...,0.196335,3.896104e-06,25.00,2.109398e+60,132.155428,2013-04-04,0.130230,25.946782,0.000515,1.488745e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5312,2023-07-27,479.500,5532.333333,14.608696,605.963768,800480.500000,6846.550725,5.998281e+05,621.224638,83826.811594,...,0.005998,5.944783e-06,6.25,2.398011e+53,29312.836826,2023-07-27,1.621684,175.826645,0.174258,5.359591e+11
5313,2023-07-28,618.500,5386.328571,14.085714,564.642857,800619.500000,6824.942857,5.802783e+05,531.842857,80769.614286,...,0.005803,5.869643e-06,6.25,2.588662e+53,29358.839335,2023-07-28,1.581364,170.362980,0.172326,5.618714e+11
5314,2023-07-29,754.500,4496.022727,11.750000,569.560606,800755.500000,6682.159091,4.889126e+05,449.606061,82856.401515,...,0.004889,5.975227e-06,6.25,2.491290e+53,29268.279610,2023-07-29,1.315909,143.096298,0.174885,4.065630e+11
5315,2023-07-30,904.500,3355.285714,9.702381,520.672619,800905.500000,6601.208333,4.179448e+05,412.202381,77184.083333,...,0.004179,5.465417e-06,6.25,2.656606e+53,29223.472673,2023-07-30,0.980531,122.137994,0.159718,3.725155e+11


In [25]:
#read the bitcoin_blockchain.db
import sqlite3
with sqlite3.connect("/home/moises/Data_BTC_thesis/bitcoin_blockchain.db") as conn:
    df_blockstats_daily = pd.read_sql_query("select * from blockchain", conn)

In [81]:
df_blockstats_daily["Date"] = pd.to_datetime(df_blockstats_daily["time"], unit='s')
df_blockstats_daily["Date"] = df_blockstats_daily["Date"].dt.date
df_blockstats_daily["Date"] = pd.to_datetime(df_blockstats_daily["Date"])
df_blockstats_daily

,index,avgfee,avgfeerate,avgtxsize,blockhash,feerate_percentiles,height,ins,maxfee,maxfeerate,...,total_out,total_size,total_weight,totalfee,txs,utxo_increase,utxo_size_inc,utxo_increase_actual,utxo_size_inc_actual,Date
0,0,0,0,0,00000000839a8e6886ab5951d76f411475428afc90947e...,"[0, 0, 0, 0, 0]",1,0,0,0,...,0,0,0,0,1,1,117,1,117,2009-01-09
1,1,0,0,0,000000006a625f06636b8bb6ac7b960a8d03705d1ace08...,"[0, 0, 0, 0, 0]",2,0,0,0,...,0,0,0,0,1,1,117,1,117,2009-01-09
2,2,0,0,0,0000000082b5015589a3fdf2d4baff403e6f0be035a5d9...,"[0, 0, 0, 0, 0]",3,0,0,0,...,0,0,0,0,1,1,117,1,117,2009-01-09
3,3,0,0,0,000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c...,"[0, 0, 0, 0, 0]",4,0,0,0,...,0,0,0,0,1,1,117,1,117,2009-01-09
4,4,0,0,0,000000009b7262315dbf071787ad3656097b892abffd1f...,"[0, 0, 0, 0, 0]",5,0,0,0,...,0,0,0,0,1,1,117,1,117,2009-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800194,994,4211,6,878,00000000000000000004d9d6648183fca80eade9244e12...,"[5, 5, 6, 6, 6]",800995,5625,507500,469,...,75267186598,1385348,3991874,6642210,1578,4769,416019,4743,413493,2023-07-31
800195,995,2163,9,398,00000000000000000004a9dec1affd8d1ba1467a5acbff...,"[5, 5, 6, 8, 13]",800996,6012,257520,1421,...,427865915720,1708303,3991678,9282582,4292,4933,374600,4919,373048,2023-07-31
800196,996,1335,6,389,00000000000000000001194fedbf6690a9d731eabdee21...,"[5, 5, 5, 6, 7]",800997,7273,635761,130,...,120743436864,2000527,3991681,6863966,5141,508,1529,502,817,2023-07-31
800197,997,5345,12,598,000000000000000000015915eab32a096622412c9b39bb...,"[6, 7, 8, 12, 18]",800998,6919,398769,588,...,564918276171,1417972,3991996,12675153,2372,803,60083,777,57099,2023-07-31


In [82]:
subsidy_daily_sum = df_blockstats_daily["subsidy"].groupby(df_blockstats_daily["Date"]).sum()
subsidy_daily_sum = pd.DataFrame(subsidy_daily_sum)
subsidy_daily_sum.columns = ["subsidy"]
subsidy_daily_sum

,subsidy
Date,
2009-01-09,70000000000
2009-01-10,305000000000
2009-01-11,465000000000
2009-01-12,470000000000
2009-01-13,615000000000
...,...
2023-07-27,86250000000
2023-07-28,87500000000
2023-07-29,82500000000


In [121]:
#get only the data from date greater than 2013-03-31 from subsidy_daily_sum_copy
#subsidy_daily_sum_copy = subsidy_daily_sum_copy[subsidy_daily_sum_copy["Date"]>="2013-03-31"]
subsidy_daily_sum_copy = (subsidy_daily_sum.cumsum()/100_000_000).copy()
subsidy_daily_sum_copy["Date"] = subsidy_daily_sum_copy.index.values
subsidy_daily_sum_copy = subsidy_daily_sum_copy[subsidy_daily_sum_copy["Date"]>="2013-03-31"]
subsidy_daily_sum_copy = subsidy_daily_sum_copy[subsidy_daily_sum_copy["Date"]<="2023-07-15"]
subsidy_daily_sum_copy["cumsum_subsidy"] = subsidy_daily_sum_copy["subsidy"]
#put values in USD
#subsidy_daily_sum_copy["subsidy"] = subsidy_daily_sum_copy["subsidy"]*df_price_btc_usd_agregate_svar["Close"]
subsidy_daily_sum_copy

,subsidy,Date,cumsum_subsidy
Date,,,
2013-03-31,10964200.00,2013-03-31,10964200.00
2013-04-01,10969150.00,2013-04-01,10969150.00
2013-04-02,10973675.00,2013-04-02,10973675.00
2013-04-03,10978050.00,2013-04-03,10978050.00
2013-04-04,10981900.00,2013-04-04,10981900.00
...,...,...,...
2023-07-11,19407556.25,2023-07-11,19407556.25
2023-07-12,19408456.25,2023-07-12,19408456.25
2023-07-13,19409206.25,2023-07-13,19409206.25


In [125]:
result = pd.DataFrame(df_price_btc_usd_agregate_svar["Close"].values*subsidy_daily_sum_copy["cumsum_subsidy"].values, index=df_price_btc_usd_agregate_svar.index.values, columns=["cumsum_subsidy"])
pd.DataFrame(result)


,cumsum_subsidy
2013-03-31,1.016860e+09
2013-04-01,1.101405e+09
2013-04-02,1.185855e+09
2013-04-03,1.408057e+09
2013-04-04,1.451318e+09
...,...
2023-07-11,5.894394e+11
2023-07-12,6.104178e+11
2023-07-13,5.882786e+11
2023-07-14,5.879442e+11


# Creation of the 2 series
volume_bitfinex/supply_btc(in USD) 

total_output(in USD)/supply_btc(in USD)

In [103]:
pd.DataFrame(subsidy_daily_sum_copy["cumsum_subsidy"])

,cumsum_subsidy
Date,
2013-03-31,10964200.00
2013-04-01,10969150.00
2013-04-02,10973675.00
2013-04-03,10978050.00
2013-04-04,10981900.00
...,...
2023-07-11,19407556.25
2023-07-12,19408456.25
2023-07-13,19409206.25


In [109]:
df_svar = df_bitfinex_volume
df_svar["cumsum_subsidy"] = subsidy_daily_sum_copy["cumsum_subsidy"]
df_svar

,Daily Volume,cumsum_subsidy
Date,,
2013-03-31,5224.401313,10964200.00
2013-04-01,8376.527478,10969150.00
2013-04-02,12996.245072,10973675.00
2013-04-03,6981.668305,10978050.00
2013-04-04,5234.882203,10981900.00
...,...,...
2023-07-11,1636.401373,19407556.25
2023-07-12,3763.115501,19408456.25
2023-07-13,2982.608992,19409206.25
